In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

transform=transforms.ToTensor()
train_data=datasets.MNIST(root='./data',train=True,download=True,transform=transform)
test_data=datasets.MNIST(root='./data',train=False,download=True,transform=transform)


train_loader=DataLoader(train_data,batch_size=64,shuffle=False)
test_loader=DataLoader(test_data,batch_size=64,shuffle=False)

class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers=nn.Sequential(
        nn.Flatten(),
        nn.Linear(28*28,256),
        nn.ReLU(),
        nn.Linear(256,128),
        nn.ReLU(),
        nn.Linear(128,10)
    )

  def forward(self,x):
    if self.layers is None:
      raise Exception("Model not initialized")
    return self.layers(x)


100%|██████████| 9.91M/9.91M [00:00<00:00, 18.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 497kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.95MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 14.0MB/s]


In [3]:
model = MLP()

In [4]:
criterion=nn.CrossEntropyLoss()

In [5]:
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [6]:
for epoch in range(5):
  for images,labels in train_loader:
    outputs=model(images)
    loss=criterion(outputs,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(f"epoch{epoch+1},Loss:{loss.item():.3f}")
print("Training complete")

epoch1,Loss:0.045
epoch2,Loss:0.042
epoch3,Loss:0.025
epoch4,Loss:0.012
epoch5,Loss:0.001
Training complete


In [8]:
correct=0
total=0
model.eval()
with torch.no_grad():
  for images,labels in test_loader:
    outputs=model(images)
    _, predicted= torch.max(outputs.data,1)
    total +=labels.size(0)
    correct +=(predicted == labels).sum().item()

print(f"Test Accuracy: {100*correct/total}")

Test Accuracy: 97.84


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP().to(device)

for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)
